In [1]:
import os
import requests
import sys
import urllib.request
from urllib.request import urlopen
import re
from pandas import DataFrame
import pandas as pd
from pandas.compat import StringIO
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import matplotlib.pyplot as plt
import json
from PIL import Image
# nuevo
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual,\
Button, HBox, VBox, IntSlider, Label, IntRangeSlider
from ipywidgets import Checkbox, RadioButtons
import random
from matplotlib import cm

In [ ]:
os.makedirs('datos',exist_ok=True)
files = ['File_1.tsv','File_10.tsv','File_2.tsv','File_3.tsv','File_5.tsv','File_6.tsv',
         'File_8.tsv','File_9.tsv']
for i in files:
    archivos = urllib.request.urlretrieve('https://raw.githubusercontent.com/eduardo1011/Bioinformatica2019/master/'+i, 'datos/'+i)
os.makedirs('img',exist_ok=True)

# Si no hay alguna ontología se descargará automáticamente

In [ ]:
modulo = urllib.request.urlretrieve('https://raw.githubusercontent.com/eduardo1011/Bioinformatica2019/master/funciones.py', 'funciones.py')
import funciones
from funciones import *

# Qué información va a ser anotada

# 1. Preparar la lista de identificadores UniProtKB
<img src="https://raw.githubusercontent.com/eduardo1011/Bioinformatica2019/master/Transcriptome_Arabidopsis.PNG" width = 80%> <br>
### https://www.g3journal.org/content/ggg/8/9/3093.full.pdf
### Condiciones, genes sobre-regulados

In [ ]:
file

In [ ]:
lista = frame()[0]
lista

### Extraer información desde UniProtKB, usando identificadores específicos
* <font color=green>Argumentos para la función</font> <font color=red>**get_UniProtKB_info0()**</font>
* ids = <**Una lista de identificadores UniProtKB**> # con esta función se actualiza la lista de IDs de UniProt
* <font color=green>Argumentos para la función</font> <font color=red>**get_UniProtKB_info1()**</font>
* ids = <**Una lista de identificadores UniProtKB**>
* <font color=green>Argumentos para la función</font> <font color=red>**get_UniProtKB_info2()**</font>
* id_organism = <**Identificador taxonómico**>

In [ ]:
# con get_UniProtKB_info0(ids = lista).Entry se obtiene una lista actualizada de identificadores de UniProt, y es la que 
#                                            se mapea en la base de datos de UniProtKB
actualizacion = get_UniProtKB_info0(ids = lista)
print('\nIdentificadores actualizados\n\nOld', '    New\n')
for index, row in actualizacion.iterrows():
    if row.ACC != row.Entry:
        print(row.ACC, row.Entry)
    else:
        continue

In [ ]:
# opción uno, descarga de información funcional desde UniProtKB de identificadores específicos

uniprotkb = get_UniProtKB_info1(ids = get_UniProtKB_info0(ids = lista).Entry)

###
# segunda opción si la anterior no funciona, descargar toda la anotación del proteoma y mapear los identificadores
# ejemplo de descarga Arabidopsis thaliana

#id_organism = 3702 
#descarga = get_UniProtKB_info2(id_organism = 3702)
#descarga.head()

#uniprotkb = DataFrame(lista, columns = ['Entry']).merge(descarga, on = 'Entry', how = 'left')
#uniprotkb
# exportar este data frame, será usado por las funciones interactivas
uniprotkb.to_csv('uniprotkb.tsv', sep = '\t', index = None)
uniprotkb.head()

### Anotación funcional depositada en UniProtKB (GO Terms)

In [ ]:
anotation = []
for index, row in uniprotkb.iterrows():
    for j in row.GO.split('; '):
        if re.search('GO:\d+', j):
            anotation.append([row.Entry, re.search('GO:\d+', j).group()])
        else:
            anotation.append([row.Entry, j])
anotation = DataFrame(anotation, columns = ['Entry', 'GO'])
anotation = anotation[anotation['GO'].str.contains('NA') == False]
anotation

# 2. Ontología

### Crear un data frame a partir del archivo go-basic.obo para hacer mapeos

In [ ]:
ontologia = ontology()
ontologia

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

print('2006 https://academic.oup.com/nar/article/34/suppl_1/D322/1132320')
print('2007 https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2238979/')
print('2009 https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2808930/')
print('2011 https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3245151/')

print('2012 https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3531070/')
print('2014 https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4383973/')
print('2016 https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5210579/')
labels = ['Sept.\n2006', 'Sept.\n2007', 'Sept.\n2009', 'Sept.\n2011',
          'Sept.\n2012', 'Sept.\n2014', 'Oct.\n2016', 'Sept.\n2018', 'Sept.\n2019']
p = [9805,13916,17069,21394,23907, 27284, 29146, 30782, ontologia[ontologia.Aspect == 'P'].count()[0]]
f = [7076,7878,8637,9062,9459, 10733, 10417, 12110, ontologia[ontologia.Aspect == 'F'].count()[0]]
c = [1574,2007,2432,2896,3050, 3758, 4022, 4386, ontologia[ontologia.Aspect == 'C'].count()[0]]
x = np.arange(len(labels))  # the label locations
width = 0.3  # the width of the bars

fig, ax = plt.subplots(figsize=(8, 4))
rects1 = ax.bar(x - width, p, width, label='P')
rects2 = ax.bar(x, f, width, label='F')
rects3 = ax.bar(x + width, c, width, label='C')

ax.set_ylabel('Terms', color='white')
#ax.set_title('Annotation update\n')
ax.set_xticks(x)
ax.tick_params(axis='x', colors='white')
ax.tick_params(axis='y', colors='white')

ax.set_yticklabels([0, 5000, 10000, 15000, 20000, 25000, 30000], color='white')
ax.set_xticklabels(labels, color='white')

ax.legend(prop=dict(weight='bold'), loc="upper left")

def autolabel(rects):
    """Attach a text label above each bar in *rects*, displaying its height."""
    for rect in rects:
        height = rect.get_height()
        ax.annotate('{}'.format(height),color='white', weight="bold",
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom')

autolabel(rects1)
autolabel(rects2)
autolabel(rects3)

plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['left'].set_color('white')
plt.gca().spines['bottom'].set_color('white')
fig.tight_layout()
plt.gca().set_facecolor('black')
ax.figure.set_facecolor('black')
#plt.savefig('Annotation_update.png', dpi = 900, bbox_inches='tight', transparent=True)
plt.show()

# 3. Seleccionar un GO Slim

### GO Slim disponibles

In [ ]:
GOSLIM

### Definimos un GO Slim

In [ ]:
slim = frame_goslim()
slim

### Explorar términos

In [ ]:
ter = interactive(explorar_terms)
ter

### Data frame "GO is_a" a partir del archivo go-basic.obo para buscar super-términos

In [ ]:
is_a = IS_A()
is_a

### Cómo se visualizan los super-términos de un GO

In [ ]:
# Ejemplo
# super_terms(term)   =>   Return a set of all super terms of term up to the most general one.
@interact
def superterminos(Term = random.choices(population=list(anotation.GO.drop_duplicates()), k=50)):
    #print('\nSuper-términos de:', Term)
    get_terms = list(super_terms(Term))
    super_t = DataFrame(get_terms, columns = ['GO'])
    def highlight_go(g):
        go = g == Term
        return ['background-color: orange' if v else '' for v in go]
    return super_t.merge(ontologia, on = 'GO', how = 'left').dropna().style.apply(highlight_go)

### Visualizar GO términos padre
##### <font color=red>'Devuelve tres salidas, la primera</font> <font color=green>[0]</font> <font color=red>contiene una descripción de nodos (GO terms) en formato JSON (JavaScript object notation) compatible con Amigo AmiGO 2 version: 2.5.12 http://amigo.geneontology.org/visualize?mode=client_amigo, la segunda</font> <font color=green>[1]</font> <font color=red>contiene una URL con extensión .png y la tercera</font> <font color=green>[2]</font> <font color=red>una URL con extensión .svg '</font>
* <font color=green>Argumentos para la función</font> <font color=red>**GO_DAG()**</font>
* go_terms = <**Una lista de GO terms**>
* term = <**Algún GO term**>

In [ ]:
Image.open(requests.get(GO_DAG(go_terms = list(super_terms(ter.result)), term = ter.result)[1], stream=True).raw)

In [ ]:
# vista en AmiGO 2
print(GO_DAG(list(super_terms(ter.result)), ter.result)[0])

## Visualizar los términos en QuickGO: GO Annotatios<br>
<img src="https://www.ebi.ac.uk/training/online/sites/ebi.ac.uk.training.online/files/resize/u13218/Screen%20Shot%202017-11-21%20at%2010.14.25-750x468.png" width = 50%> <br>
The QuickGO REST API provides access to key biological data from QuickGO and GOA.<br>
https://www.ebi.ac.uk/QuickGO/api/index.html

In [ ]:
lista_sup = re.sub(':', '%3A', '%2C'.join(list(super_terms(ter.result))))
quickgo_url = 'https://www.ebi.ac.uk/QuickGO/services/ontology/go/terms/%7Bids%7D/chart?ids='
Image.open(requests.get(quickgo_url+lista_sup, stream=True).raw)

# 4. GO Slim

In [ ]:
dfsup = mapping_super_terms(list_gos = anotation.GO.drop_duplicates())

In [ ]:
dfsup.head()

In [ ]:
asp = {'P':'Biological Process', 'F':'Molecular Function', 'C':'Cellular Component'}
df2 = anotation.merge(dfsup, on = 'GO', how = 'inner')
df3 = df2.merge(ontologia, on = 'GO', how = 'left')
df3.columns = ['Entry', 'Original GO', 'is_a','Original Term', 'Original Aspect']
df4 = df3.merge(is_a[['is_a','Term','Aspect']], on = 'is_a', how = 'left').drop_duplicates().reset_index(drop = True).dropna()
df4 = df4.rename(columns={'is_a':'GO'}) # is_a se convierte en GO, para mapearlos con el GO Slim seleccionado
slim['ASPECT'] = [asp[i] for i in slim.Aspect]
GO_Slim = slim.merge(df4, on = ['GO', 'Term', 'Aspect'],
                     how = 'left').drop_duplicates().reset_index(drop = True).dropna().reset_index(drop = True)
etiquetas = []    
for i in GO_Slim.Term:
    if len(i.split(' ')) <= 3:
        etiquetas.append(re.sub(' ', '\n', i))
    else:
        etiquetas.append(''.join(i.split(' ')[0])+'\n'+''.join(i.split(' ')[1])+''.join(i.split(' ')[2])+'...') # +'\n'+i.split(' ')[2]
GO_Slim['Short_Term'] = etiquetas 
print('\n\nColumnas del GO Slim: [GO, Term, Aspect, Short_Term]')
# exportar este data frame, será usado por las funciones interactivas
GO_Slim.to_csv('GO_Slim.tsv', sep = '\t', index = None)
GO_Slim

# 5. Visualizaciones gráficas <br>

In [ ]:
categorias = DataFrame(GO_Slim[['ASPECT', 'Original Aspect', 'Aspect', 'Entry']].drop_duplicates().groupby(['ASPECT', 'Aspect', 'Original Aspect']).Entry.count()).reset_index()
categorias

### creación de un diccionario

In [ ]:
diccionario = {'numeros':[1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
            'letras':['a','b','c','d','e','f','g','h','i','j']}
diccionario

### extraer las claves/llaves del diccionario

In [ ]:
diccionario.keys()

### extraer los valores del diccionario

In [ ]:
diccionario.values()

## Diagrama de Venn

In [ ]:
intersecciones = {}
for i in ['P', 'F', 'C']:
    intersecciones[i] = set(GO_Slim[GO_Slim.Aspect == i].Entry)
    

In [ ]:
plt.subplots(figsize=(6,6))
venn3_plot(set1 = intersecciones['P'], set2 = intersecciones['F'], set3 = intersecciones['C'])

# ----------------------------------------------------------------------------------------------------------

## Biological Process

In [ ]:
columnas = ['GO', 'Entry', 'Aspect', 'Term', 'ASPECT', 'Short_Term']

In [ ]:
pro1 = GO_Slim[columnas].groupby(['Aspect']).get_group('P').drop_duplicates()
proceso = DataFrame(pro1.drop_duplicates().groupby(['GO','Term', 'Short_Term']).Entry.count()).reset_index()
pro = proceso.sort_values(by ='Entry',ascending=False).reset_index(drop=True)
pro.head()

## graficos individuales

In [ ]:
plt.subplots(figsize=(6,6))
barras(df = pro)

### <font color=red>Ejemplo = Control de más parámetros</font>

In [ ]:
plt.subplots(figsize=(6,6))
barras(df = pro, column = 1, dim = 111, title = 'Biological Process', row_num = 15, color = 'C1',
           size_x = 15, size_y = 15, xlabel = 20, size_title = 25, size_bartxt = 12)

In [ ]:
# graficos individuales
plt.subplots(figsize=(6,6))
pastel(df = pro)

### <font color=red>Ejemplo = Control de más parámetros</font>

In [ ]:
plt.subplots(figsize=(6,6))
pastel(df = pro, column = 1, dim = 111, title = 'Biological Process', row_num = 7, angle = 0,
           size_text = 15, size_title = 20)

## Visualizar los términos en AmiGO 2: Visualize
##### <font color=red>'Devuelve tres salidas, la primera</font> <font color=green>[0]</font> <font color=red>contiene una descripción de nodos (GO terms) en formato JSON (JavaScript object notation) compatible con Amigo AmiGO 2 version: 2.5.12 http://amigo.geneontology.org/visualize?mode=client_amigo, la segunda</font> <font color=green>[1]</font> <font color=red>contiene una URL con extensión .png y la tercera</font> <font color=green>[2]</font> <font color=red>una URL con extensión .svg '</font>
* <font color=green>Argumentos para la función</font> <font color=red>**GO_DAG()**</font>
* go_terms = <**Una lista de GO terms**>
* term = <**Algún GO term**>

In [ ]:
Image.open(requests.get(GO_DAG(go_terms = list(pro.GO.head(20)),
                               term = list(pro.GO.head(20))[10])[1], stream=True).raw)

In [ ]:
# copiar la siguiente cadena en el sitio del enlace
#print((GO_DAG(list(pro.GO.head(20)), list(pro.GO.head(20))[10]))[0])

## Visualizar los términos en QuickGO: GO Annotatios<br>
<img src="https://www.ebi.ac.uk/training/online/sites/ebi.ac.uk.training.online/files/resize/u13218/Screen%20Shot%202017-11-21%20at%2010.14.25-750x468.png" width = 50%> <br>
The QuickGO REST API provides access to key biological data from QuickGO and GOA.<br>
https://www.ebi.ac.uk/QuickGO/api/index.html

In [ ]:
lista_sup = re.sub(':', '%3A', '%2C'.join(list(pro.GO.head(20))))
quickgo_url = 'https://www.ebi.ac.uk/QuickGO/services/ontology/go/terms/%7Bids%7D/chart?ids='
Image.open(requests.get(quickgo_url+lista_sup, stream=True).raw)

# Visualización del grado de interacciones con redes (NetworkX)
https://networkx.github.io/documentation/networkx-1.9/index.html

* <font color=green>Argumentos para la función</font> <font color=red>**net_plot()**</font>
* df = <**Data frame con tres columnas: ejemplo GO \t Entry \t Aspect \t Term**>
* label = <**Mostrar etiqueta: ejemplo 'label' o 'none'**>
* diam_nodos = <**Ajustar el diámetro de los nodos: ejemplo 10**>
* espe_edges = <**Ajustar el espesor de los vértices: ejemplo 0.2**>
* inter = <**Grado de interacción: ejemplo nodos con más de 10 interacciones (proteínas o genes que comparten)**>
* color_inter_min = <**Color de los vértices con menor interacción**>
* color_inter_max = <**Color de los vértices con mayor interacción**>
* edge_alpha_min = <**Nivel de transparencia en el color de los vértices de menor interacción**>
* edge_alpha_max = <**Nivel de transparencia en el color de los vértices de mayor interacción**>
* k_num = <**Distancia óptima entre los nodos (lo aplica al algoritmo de Fruchterman-Reingold)**><br>
https://networkx.github.io/documentation/networkx-1.9/reference/generated/networkx.drawing.layout.spring_layout.html
* color_nodo = <**Color de los nodos**>
* node_alpha = <**Nivel de transparencia en el color de los nodos**><br><br>

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt

In [ ]:
plt.subplots(figsize=(7,7))
net_plot(df = pro1)

### <font color=red>Ejemplo = Control de más parámetros</font>

In [ ]:
plt.subplots(figsize=(12,12))

net_plot(df = pro1,
         label = 'label',
         layout = 'Spring',
         column = 2,
         label_size = 10,
         diam_nodos = 20,
         espe_edges = 0.2,
         inter = 50,
         color_inter_min = 'darkgreen',
         color_inter_max = 'blue',
         edge_alpha_min = 0.3,
         edge_alpha_max = 0.7,
         k_num = 6,
         color_nodo = 'red',
         node_alpha = 0.7)

In [ ]:
pro1[['GO','Entry','Aspect','Term']].drop_duplicates().groupby(['GO','Term','Aspect'])\
.Entry.count().reset_index().sort_values(by ='Entry',ascending=False).reset_index(drop=True).head(10)

# Visualización del grado de interacciones con Heatmaps
https://seaborn.pydata.org/generated/seaborn.heatmap.html

* <font color=green>Argumentos para la función</font> <font color=red>**heatmap_plot()**</font>
* df = <**Data frame con tres columnas: ejemplo GO\tEntry\tAspect\tTerm\tASPECT\tShort_Term**>
* size_plot = <**Tamaño del gráfico**>
* label_x = <**GO terms o Terms como etiqueta**>
* label_y = <**GO terms o Terms como etiqueta**>
* xticks_size = <**Tamaño de etiqueta eje x**>
* yticks_size = <**Tamaño de etiqueta eje y**>
* ylabel_size = <**Tamaño de legenda**><br><br>

In [ ]:
plt.subplots(figsize=(10,10))
heatmap_plot(df = pro1)

### <font color=red>Ejemplo = Control de más parámetros</font>

In [ ]:
plt.subplots(figsize=(10,10))
# colors (https://matplotlib.org/examples/color/colormaps_reference.html)
heatmap_plot(df = pro1, label_x = 'Term', label_y = 'Term',
             xticks_size = 9, yticks_size = 9, ylabel_size = 10, colors='cividis',)

# ----------------------------------------------------------------------------------------------------------

<img src="https://raw.githubusercontent.com/eduardo1011/Bioinformatica2019/master/activity.png" width = 100%>

# ----------------------------------------------------------------------------------------------------------

## Molecular Function

In [ ]:
fun1 = GO_Slim[columnas].groupby(['Aspect']).get_group('F')
funcion = DataFrame(fun1.groupby(['GO','Term', 'Short_Term']).Entry.count()).reset_index()
fun = funcion.sort_values(by ='Entry',ascending=False).reset_index(drop=True)
fun.head()

# ----------------------------------------------------------------------------------------------------------

## Cellular Component

In [ ]:
com1 = GO_Slim[columnas].groupby(['Aspect']).get_group('C')
componente = DataFrame(com1.groupby(['GO','Term', 'Short_Term']).Entry.count()).reset_index()
com = componente.sort_values(by ='Entry',ascending=False).reset_index(drop=True)
com.head()

# ----------------------------------------------------------------------------------------------------------

# Múltiples gráficos

In [ ]:
# Múltiples gráficos
#https://matplotlib.org/users/dflt_style_changes.html
#1f77b4
#ff7f0e
#2ca02c

# funcion(df, column, positions, title, rows, size xlabel, size ylabel, size xtitle, size title, size bar values)

barras(pro,1,241,'Biological Process',15,'tan',5, 5, 10, 15, 7)
barras(fun,1,233,'Molecular Function',15,'tan',5, 5, 10, 15, 7)
barras(com,1,234,'Cellular Component',15,'tan',5, 5, 10, 15, 7)
pastel(categorias,0,236,'Categories',len(categorias),60,10,15)
plt.subplots_adjust(top=0.92, bottom=-0.2, left=0, right=1, hspace=0.5,
                    wspace=0)

# ----------------------------------------------------------------------------------------------------------

In [3]:
os.listdir()

['.ipynb_checkpoints',
 '1_ANOTACION_2019.ipynb',
 '2_BLAST_ANOTACION_2019.ipynb',
 'Automatic_Notebook.ipynb',
 '__pycache__']

In [2]:
import os

In [ ]:
'1_ANOTACION_2019.ipynb', '2_BLAST_ANOTACION_2019.ipynb', 'Automatic_Notebook.ipynb'